In [1]:
### voting

In [2]:
###Carga de librerias, lectura del dataset

In [3]:
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn import tree

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
#Modificar dimensiones predeterminadas de visibilidad de columnas y filas
x = 52
pd.set_option('display.max_columns', x) #Maximo de columnas a ver
pd.set_option('display.max_rows', x) #Maximo de Filas a ver

In [6]:
path_dataset = 'train.csv'
train = pd.read_csv(path_dataset)

In [7]:
path_dataset = 'test.csv'
test = pd.read_csv(path_dataset)

In [8]:
train.head()

,Opportunity_ID,Region,Billing_Country,Account_Type,Opportunity_Type,Account_Owner,Account_Name,Conversión_USD,Delivery_Quarter,TRF,Stage,promedio_preciov_region,promedio_preciov_paises,promedio_preciov_Account,promedio_preciov_Oportunity,promedio_preciov_vendedor,Wilsonxproporcion_region,Wilsonxproporcion_paises,Wilsonxproporcion_vendedor
0,0,EMEA,Netherlands,Account_Type_2,Opportunity_Type_1,Person_Name_51,Account_Name_619,6305214,Q2,10,Closed Lost,246888.239174,0.000000,0.000000,0.000000,0.000000,0.514817,0.513767,0.325851
1,1,EMEA,Netherlands,Account_Type_2,Opportunity_Type_1,Person_Name_51,Account_Name_619,57673,Q1,0,Closed Won,246888.239174,177583.146032,125034.170409,133491.432403,85165.046512,0.514817,0.513767,0.325851
2,2,Americas,United States,Account_Type_5,Opportunity_Type_1,Person_Name_64,Account_Name_1794,83865,Q1,0,Closed Won,422783.847183,432143.813675,958150.887348,133491.432403,75357.668421,0.445687,0.448366,0.616299
3,3,Americas,United States,Account_Type_5,Opportunity_Type_19,Person_Name_8,Account_Name_1201,7421881,Q1,14,Closed Lost,422783.847183,0.000000,0.000000,0.000000,0.000000,0.445687,0.448366,0.561500
4,4,Americas,United States,Account_Type_5,Opportunity_Type_19,Person_Name_8,Account_Name_1201,13357192,Q1,25,Closed Lost,422783.847183,0.000000,0.000000,0.000000,0.000000,0.445687,0.448366,0.561500


In [9]:
test.head()

,Opportunity_ID,Region,Billing_Country,Account_Type,Opportunity_Type,Account_Owner,Account_Name,Conversión_USD,Delivery_Quarter,TRF,promedio_preciov_region,promedio_preciov_paises,promedio_preciov_Account,promedio_preciov_Oportunity,promedio_preciov_vendedor,Wilsonxproporcion_region,Wilsonxproporcion_paises,Wilsonxproporcion_vendedor
0,10689,EMEA,Germany,Account_Type_0,Opportunity_Type_8,Person_Name_13,Account_Name_978,194006,Q2,0,246888.239174,325954.165775,116533.133558,87111.379883,278450.516274,0.514817,0.445687,0.639475
1,10689,EMEA,Germany,Account_Type_0,Opportunity_Type_8,Person_Name_13,Account_Name_978,94018,Q2,0,246888.239174,325954.165775,116533.133558,87111.379883,278450.516274,0.514817,0.445687,0.639475
2,10689,EMEA,Germany,Account_Type_0,Opportunity_Type_8,Person_Name_13,Account_Name_978,151334,Q2,0,246888.239174,325954.165775,116533.133558,87111.379883,278450.516274,0.514817,0.445687,0.639475
3,10690,EMEA,Germany,Account_Type_0,Opportunity_Type_8,Person_Name_13,Account_Name_978,291009,Q2,1,246888.239174,325954.165775,116533.133558,87111.379883,278450.516274,0.514817,0.445687,0.639475
4,10690,EMEA,Germany,Account_Type_0,Opportunity_Type_8,Person_Name_13,Account_Name_978,188037,Q2,0,246888.239174,325954.165775,116533.133558,87111.379883,278450.516274,0.514817,0.445687,0.639475


In [10]:
###Preprocesamiento

In [11]:
train['Stage']= (train['Stage'] == 'Closed Won') +0

In [12]:
for column in train.columns:
    if train[column].dtype == 'int32':
        train[column] = train[column].astype('float64')
print(train.dtypes)

Opportunity_ID                   int64
Region                          object
Billing_Country                 object
Account_Type                    object
Opportunity_Type                object
Account_Owner                   object
Account_Name                    object
Conversión_USD                   int64
Delivery_Quarter                object
TRF                              int64
Stage                          float64
promedio_preciov_region        float64
promedio_preciov_paises        float64
promedio_preciov_Account       float64
promedio_preciov_Oportunity    float64
promedio_preciov_vendedor      float64
Wilsonxproporcion_region       float64
Wilsonxproporcion_paises       float64
Wilsonxproporcion_vendedor     float64
dtype: object


In [13]:
### Target meaning encoding

In [14]:
# Import KFold
from sklearn.model_selection import KFold

In [15]:
def test_mean_target_encoding(train, test, target, categorical, alpha=5):
    # Calculate global mean on the train data
    global_mean = train[target].mean()
    
    # Group by the categorical feature and calculate its properties
    train_groups = train.groupby(categorical)
    category_sum = train_groups[target].sum()
    category_size = train_groups.size()
    
    # Calculate smoothed mean target statistics
    train_statistics = (category_sum + global_mean * alpha) / (category_size + alpha)
   
    # Apply statistics to the test data and fill new categories
    test_feature = test[categorical].map(train_statistics).fillna(global_mean)
    return test_feature.values

In [16]:
def train_mean_target_encoding(train, target, categorical, alpha=5):
    # Create 5-fold cross-validation
    kf = KFold(n_splits=5, random_state=123, shuffle=True)
    train_feature = pd.Series(index=train.index)
    
    # For each folds split
    for train_index, test_index in kf.split(train):
        cv_train, cv_test = train.iloc[train_index], train.iloc[test_index]
      
        # Calculate out-of-fold statistics and apply to cv_test
        cv_test_feature = test_mean_target_encoding(cv_train, cv_test, target, categorical, alpha)
        
        # Save new feature for this particular fold
        train_feature.iloc[test_index] = cv_test_feature       
    return train_feature.values

In [17]:
def mean_target_encoding(train, test, target, categorical, alpha=5):
  
    # Get the train feature
    train_feature = train_mean_target_encoding(train, target, categorical, alpha)
  
    # Get the test feature
    test_feature = test_mean_target_encoding(train, test, target, categorical, alpha)
    
    # Return new features to add to the model
    return train_feature, test_feature

In [18]:
columns = ['Region', 'Billing_Country', 'Account_Type','Opportunity_Type', 'Account_Owner', 'Account_Name','Delivery_Quarter']
for column in columns:     
    train[column], test[column] = mean_target_encoding(train=train,
                                                                test=test,
                                                                target='Stage',
                                                                categorical=column,
                                                                alpha=10)

In [19]:
###Seleccion de Features

In [20]:
train.columns

Index(['Opportunity_ID', 'Region', 'Billing_Country', 'Account_Type',
       'Opportunity_Type', 'Account_Owner', 'Account_Name', 'Conversión_USD',
       'Delivery_Quarter', 'TRF', 'Stage', 'promedio_preciov_region',
       'promedio_preciov_paises', 'promedio_preciov_Account',
       'promedio_preciov_Oportunity', 'promedio_preciov_vendedor',
       'Wilsonxproporcion_region', 'Wilsonxproporcion_paises',
       'Wilsonxproporcion_vendedor'],
      dtype='object')

In [21]:
features = ['Region', 'Billing_Country', 'Account_Type',
       'Opportunity_Type', 'Account_Owner', 'Account_Name', 'Conversión_USD',
       'Delivery_Quarter', 'TRF', 'Wilsonxproporcion_region', 'Wilsonxproporcion_paises',
       'Wilsonxproporcion_vendedor']

In [22]:
### Modelos

In [23]:
X = train.drop(['Stage'], axis=1)
y = train['Stage']

In [24]:
###Separando en conjuntos de entrenamiento y de testeo¶

In [25]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.2, random_state=123)

In [26]:
###Entrenando los modelos

In [27]:
# entrenar el modelo en esta celda
xgb_clf=xgb.XGBClassifier()

In [28]:
param_grid = [{    
        'objective': ['binary:logistic'],
        'n_estimators': [50, 200, 1000],
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [3, 4, 5],
        'colsample_bytree': [0.3, 0.7], 
        'learning_rate':[0.01],
        'reg_alpha' : [0.3]        
                     
                     }]

In [29]:
from sklearn.model_selection import GridSearchCV

from sklearn.model_selection import cross_val_score

In [30]:
grid_search = GridSearchCV(xgb_clf, param_grid= param_grid, cv = 5, scoring="neg_log_loss", return_train_score=True).fit(X_train[features], y_train)

[01:35:38] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[01:35:39] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[01:35:39] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[01:35:39] WARNING: C:/Users/Administrator/workspace/xgboost-win64_rel

In [32]:
y_pred=grid_search.predict_proba(X_test[features])

In [34]:
y_pred

array([[0.19957131, 0.8004287 ],
       [0.96423274, 0.03576725],
       [0.95472157, 0.04527846],
       ...,
       [0.49888092, 0.5011191 ],
       [0.4493404 , 0.5506596 ],
       [0.27217174, 0.72782826]], dtype=float32)

In [38]:
# Import log_loss from sklearn
from sklearn.metrics import log_loss

In [39]:
print('Sklearn LogLoss: {:.5f}'.format(log_loss(y_test, y_pred)))

Sklearn LogLoss: 0.44121


In [35]:
optimised_xgb= grid_search.best_estimator_

In [36]:
optimised_xgb

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.7, gamma=1, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.01, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=1000, n_jobs=4, num_parallel_tree=1, random_state=0,
              reg_alpha=0.3, reg_lambda=1, scale_pos_weight=1, subsample=0.8,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [42]:
y__opt_pred=optimised_xgb.predict_proba(X_test[features])

In [47]:
import lightgbm as lgb

In [48]:
lgb_clf = lgb.LGBMClassifier()
param_dist = [{"max_depth": [5,10,15],
              "learning_rate" : [0.1,0.15,0.3],
              "num_leaves": [32,150,200],
              "n_estimators": [300,400,1000], 
               'objective' : ['binary']
             }]

In [49]:
grid_search1 = GridSearchCV(lgb_clf, param_grid=param_dist, cv = 3, scoring="neg_log_loss", return_train_score=True).fit(X_train[features], y_train)

In [50]:
optimised_lgb= grid_search1.best_estimator_

In [55]:
optimised_lgb

LGBMClassifier(max_depth=5, n_estimators=300, num_leaves=32, objective='binary')

In [53]:
y_pred=grid_search1.predict_proba(X_test[features])

In [54]:
print('Sklearn LogLoss: {:.5f}'.format(log_loss(y_test, y_pred)))

Sklearn LogLoss: 0.43709


In [56]:
from sklearn.metrics import accuracy_score
from sklearn.ensemble import VotingClassifier

In [60]:
voting_clf = VotingClassifier(
estimators=[('xgb', xgb_clf), ('lgb', lgb_clf)],voting='soft')

In [61]:
for clf in (xgb_clf, lgb_clf, voting_clf):
    clf.fit(X_train[features], y_train)
    y_pred = clf.predict(X_test[features])
    print(clf.__class__.__name__, log_loss(y_test, y_pred))

[05:27:55] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
XGBClassifier 7.123893940150142
LGBMClassifier 7.072946198020504
[05:27:56] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
VotingClassifier 7.08313433081215


In [62]:
preds_test= voting_clf.predict_proba(test[features])

In [64]:
preds_test=preds_test[:, 1]

In [65]:
test['Target']=preds_test

In [66]:
test['Target'].shape

(2551,)

In [68]:
sub= test[['Opportunity_ID','Target']]

In [69]:
sub1=sub.groupby(['Opportunity_ID'], as_index=False).mean()

In [70]:
sub1[['Opportunity_ID', 'Target']].to_csv('submission12.csv', index=False)

In [71]:
test = test.drop(['Target'], axis=1)